In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/asl-dataset/asl_dataset/n/hand2_n_dif_seg_3_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand1_n_left_seg_5_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand2_n_top_seg_4_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand1_n_dif_seg_2_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand2_n_top_seg_1_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand3_n_dif_seg_1_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand2_n_dif_seg_2_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand2_n_top_seg_3_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand5_n_dif_seg_4_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand1_n_right_seg_5_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand1_n_left_seg_1_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand2_n_right_seg_4_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand1_n_right_seg_3_cropped.jpeg
/kaggle/input/asl-dataset/asl_dataset/n/hand4_n_bot_seg_2_cropped.jpe

In [3]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 47.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is inc

In [4]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import mediapipe as mp
import matplotlib.pyplot as plt

# Function to detect landmarks
def detect_landmarks(image):
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils

    # Initialize MediaPipe Hands
    with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.7) as hands:
        # Add a border to the image
        border_color = (0, 0, 0)  # Black color
        border_width = 250
        image_with_border = cv2.copyMakeBorder(image, border_width, border_width, border_width, border_width, cv2.BORDER_CONSTANT, value=border_color)

        # Convert image with border to RGB (MediaPipe uses RGB format)
        image_with_border_rgb = cv2.cvtColor(image_with_border, cv2.COLOR_BGR2RGB)

        # Process image
        results = hands.process(image_with_border_rgb)

        # Placeholder for landmarks if no hands are detected
        placeholder_landmarks = None

        # Check if hands are detected
        if results.multi_hand_landmarks:
            # Draw landmarks on the image
            annotated_image = image_with_border.copy()
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Crop the border out before returning the image
            height, width, _ = image.shape
            cropped_image = annotated_image[border_width:border_width+height, border_width:border_width+width]
            return cropped_image
        else:
            print("No hands detected in the image.")
            return image 

# Function to preprocess image
def preprocess_image(image, target_size=(224, 224)):
    # Preprocess the image
    preprocessed_image = cv2.resize(image, target_size)
    preprocessed_image = preprocessed_image / 255.0
    preprocessed_image = (preprocessed_image * 255).astype(np.uint8)
    
    if np.random.rand() > 0.5:
        preprocessed_image = cv2.flip(preprocessed_image, 1)
    if np.random.rand() > 0.5:
        preprocessed_image = cv2.flip(preprocessed_image, 0)
    if np.random.rand() > 0.5:
        angle = np.random.uniform(-15, 15)
        M = cv2.getRotationMatrix2D((target_size[0] // 2, target_size[1] // 2), angle, 1)
        preprocessed_image = cv2.warpAffine(preprocessed_image, M, (target_size[0], target_size[1]))
    if np.random.rand() > 0.5:
        zoom_factor = np.random.uniform(0.9, 1.1)
        preprocessed_image = cv2.resize(preprocessed_image, None, fx=zoom_factor, fy=zoom_factor)
        if zoom_factor < 1.0:
            pad_h = (target_size[0] - preprocessed_image.shape[0]) // 2
            pad_w = (target_size[1] - preprocessed_image.shape[1]) // 2
            preprocessed_image = cv2.copyMakeBorder(preprocessed_image, pad_h, pad_h, pad_w, pad_w, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        else:
            crop_h = (preprocessed_image.shape[0] - target_size[0]) // 2
            crop_w = (preprocessed_image.shape[1] - target_size[1]) // 2
            preprocessed_image = preprocessed_image[crop_h:crop_h+target_size[0], crop_w:crop_w+target_size[1]]
    if np.random.rand() > 0.5:
        translation = np.random.uniform(-10, 10, 2)
        M = np.float32([[1, 0, translation[0]], [0, 1, translation[1]]])
        preprocessed_image = cv2.warpAffine(preprocessed_image, M, (target_size[0], target_size[1]))
    preprocessed_image = cv2.GaussianBlur(preprocessed_image, (5, 5), 0)
    
    return preprocessed_image

# Function to load data
def load_data(directory, target_size=(224, 224)):
    images = []
    labels = []
    
    # List all subdirectories (classes) in the dataset directory
    classes = os.listdir(directory)
    
    for folder in classes:
        folder_path = os.path.join(directory, folder)
        
        # Skip non-directories
        if not os.path.isdir(folder_path):
            continue
        
        # Get the label from the folder name
        label = folder
        
        # Iterate over all images in the folder
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            
            # Read the image
            image = cv2.imread(image_path)
            if image is not None:
                # Apply hand landmark detection and preprocessing
                preprocessed_image = detect_landmarks(image)
                if preprocessed_image is None:
                    # If no hands detected, preprocess without hand landmarks
                    preprocessed_image = preprocess_image(image)
                else:
                    # If hands detected, preprocess the annotated image
                    preprocessed_image = preprocess_image(preprocessed_image)
                
                # Resize the preprocessed image to target size
                preprocessed_image = cv2.resize(preprocessed_image, target_size)

                # Append the image and its corresponding label
                images.append(preprocessed_image)
                labels.append(label)
    
    # Convert lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels

In [5]:
import warnings
import logging
import tensorflow as tf

# # Set TensorFlow logging level
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0 = all logs, 1 = filter INFO, 2 = filter WARNING, 3 = filter ERROR

# # Suppress warnings from the warnings module
# warnings.filterwarnings('ignore')

# # Suppress absl-py logging
# logging.getLogger('absl').setLevel(logging.ERROR)

# # Optionally, if using TensorFlow Lite Interpreter
# # Set verbosity level for TensorFlow Lite logs (only works with TensorFlow 2.3 and later)
# if hasattr(tf, 'lite'):
#     from tensorflow.lite.python.interpreter import Interpreter

#     # Custom Interpreter class to suppress TFLite logs
#     class QuietInterpreter(Interpreter):
#         def __init__(self, *args, **kwargs):
#             super().__init__(*args, **kwargs)
#             self._delegate_options.update({'--log-level': 'FATAL'})
            
dataset_dir = "/kaggle/input/asl-dataset/asl_dataset/asl_dataset"
X, y = load_data(dataset_dir)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1725869010.059483     117 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869010.090516     117 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869010.101502     119 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/opt/conda/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
W0000 00:00:1725869010.176007     126 inference_feedback_ma

No hands detected in the image.


W0000 00:00:1725869013.162132     294 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869013.183638     294 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869013.228229     298 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869013.240967     296 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869013.289914     302 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869013.308832     302 inference_feedback_manager.cc:114] Feedback manager 

No hands detected in the image.


W0000 00:00:1725869013.850859     337 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869013.870290     337 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869013.923850     343 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869013.945492     343 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869013.991271     346 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869014.009689     346 inference_feedback_manager.cc:114] Feedback manager 

...

W0000 00:00:1725869158.398568    9314 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869158.443953    9318 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869158.461669    9317 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869158.510940    9324 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869158.524356    9324 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869158.571197    9326 inference_feedback_manager.cc:114] Feedback manager 

No hands detected in the image.


W0000 00:00:1725869160.150246    9425 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869160.201555    9431 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869160.219499    9432 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869160.267754    9436 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869160.286759    9435 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869160.338090    9438 inference_feedback_manager.cc:114] Feedback manager 

No hands detected in the image.


W0000 00:00:1725869165.768250    9782 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869165.781522    9784 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869165.828870    9786 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869165.847292    9786 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869165.898063    9790 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725869165.911526    9789 inference_feedback_manager.cc:114] Feedback manager 

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
encoder = OneHotEncoder(sparse_output = False)
y_encoded = encoder.fit_transform(y.reshape(-1,1))
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size =0.2, random_state= 42)

In [7]:
X = X / 255.0
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)  # 70% train, 30% temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Split temp into 50% validation, 50% test
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

def data_generator(images, labels, batch_size, sequence_length):
    while True:
        for i in range(0, len(images), batch_size):
            batch_images = []
            batch_labels = []
            for j in range(i, i + batch_size):
                if j >= len(images):
                    break
                image_sequence = []
                for k in range(sequence_length):
                    image_sequence.append(images[j])  # For simplicity, using the same image for the sequence
                batch_images.append(image_sequence)
                batch_labels.append(labels[j])
            yield np.array(batch_images), np.array(batch_labels)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, LSTM, BatchNormalization, Dropout, Lambda
from tensorflow.keras.applications import ResNet50

num_classes = len(label_encoder.classes_)

# Define input shape and sequence length
input_shape = (224, 224, 3)
sequence_length = 10  # Example sequence length

# Pretrained model for feature extraction
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Define the CNN-LSTM model
input_layer = Input(shape=(sequence_length, *input_shape))

# Process each frame in the sequence
def process_frame(x):
    features = []
    for i in range(sequence_length):
        frame = x[:, i, :, :, :]
        feature = base_model(frame)
        feature = Flatten()(feature)
        features.append(feature)
    return tf.stack(features, axis=1)

x = Lambda(process_frame)(input_layer)
x = LSTM(256, return_sequences=False)(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output_layer = Dense(len(set(y_train)), activation='softmax')(x)  # Assuming the number of classes is len(set(y_train))
cnn_lstm_resnet_model = Model(inputs=input_layer, outputs=output_layer)
cnn_lstm_resnet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                              loss='sparse_categorical_crossentropy', 
                              metrics=['accuracy'])

# Print the model summary
cnn_lstm_resnet_model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 10, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 10, 100352)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 256)            │   103,023,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 36)             │         9,252 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 103,033,892 (393.04 MB)

 Trainable params: 103,033,380 (393.04 MB)

 Non-trainable params: 512 (2.00 KB)

In [9]:
from tensorflow.keras.callbacks import EarlyStopping
batch_size = 16
patience =5
train_gen = data_generator(X_train, y_train, batch_size, sequence_length)
val_gen = data_generator(X_val, y_val, batch_size, sequence_length)
test_gen = data_generator(X_test, y_test, batch_size, sequence_length)


# Calculate steps per epoch and validation steps
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size
early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
# Train the model
history_cnn_lstm_resnet = cnn_lstm_resnet_model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=100, validation_data=val_gen, validation_steps=validation_steps,callbacks=[early_stopping])

test_loss, test_accuracy = cnn_lstm_resnet_model.evaluate(test_gen, steps=len(X_test) // batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

Epoch 1/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 216s 911ms/step - accuracy: 0.2004 - loss: 3.4265 - val_accuracy: 0.3261 - val_loss: 2.5850
Epoch 2/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 81s 735ms/step - accuracy: 0.7185 - loss: 1.0969 - val_accuracy: 0.7147 - val_loss: 1.2755
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 80s 731ms/step - accuracy: 0.8704 - loss: 0.5824 - val_accuracy: 0.7562 - val_loss: 0.9089
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 80s 733ms/step - accuracy: 0.9300 - loss: 0.3638 - val_accuracy: 0.7978 - val_loss: 0.7150
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 80s 732ms/step - accuracy: 0.9778 - loss: 0.1794 - val_accuracy: 0.8089 - val_loss: 0.6131
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 80s 731ms/step - accuracy: 0.9901 - loss: 0.1067 - val_accuracy: 0.8449 - val_loss: 0.5569
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 80s 733ms/step - accuracy: 0.9941 - loss: 0.0778 - val_accuracy: 0.8366 - val_loss: 0.5377
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 80s 732ms/step - accuracy: 0.9942 

In [10]:
# Save the entire model
cnn_lstm_resnet_model.save('cnn_lstm_resnet_model.h5')


In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import register_keras_serializable

def process_frame(x):
    features = []
    for i in range(sequence_length):
        frame = x[:, i, :, :, :]
        feature = base_model(frame)
        feature = Flatten()(feature)
        features.append(feature)
    return tf.stack(features, axis=1)

# Load the saved model
loaded_model = load_model('cnn_lstm_resnet_model.h5', custom_objects={'process_frame': process_frame})


In [12]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

def extract_frames_from_video(video_path, sequence_length, input_shape):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or count >= sequence_length:
            break
        # Resize frame to the expected input shape (e.g., 224x224)
        frame_resized = cv2.resize(frame, (input_shape[0], input_shape[1]))
        frames.append(frame_resized)
        count += 1

    cap.release()

    # Check if frames were captured; if not, handle the empty list
    if len(frames) == 0:
        print(f"Error: No frames captured from the video {video_path}.")
        return None

    # Ensure we have exactly `sequence_length` frames; if not, pad with the last frame
    while len(frames) < sequence_length:
        frames.append(frames[-1])

    # Convert to NumPy array and normalize
    frames = np.array(frames).astype('float32') / 255.0
    return frames

# Define your parameters
video_paths = ['/kaggle/input/test-case-1/Dataset 1.mp4']  # Example list of video paths
sequence_length = 10  # Same sequence length used for training
input_shape = (224, 224, 3)  # Expected shape of each frame

# Prepare the video sequences
video_sequences = []

for video_path in video_paths:
    frames = extract_frames_from_video(video_path, sequence_length, input_shape)
    if frames is not None:  # Only add to the sequence if frames are captured
        video_sequences.append(frames)

# Ensure we have sequences to predict
if len(video_sequences) == 0:
    print("Error: No valid video sequences found for prediction.")
else:
    # Convert to numpy array with the shape (batch_size, sequence_length, height, width, channels)
    video_sequences = np.array(video_sequences)

    # Predict using the trained model
    predictions = cnn_lstm_resnet_model.predict(video_sequences)

    # Convert predictions to class labels
    predicted_classes = np.argmax(predictions, axis=1)

    # Print the predicted classes for each video
    for i, video_path in enumerate(video_paths):
        print(f"Prediction for {video_path}: {predicted_classes[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step
Prediction for /kaggle/input/test-case-1/Dataset 1.mp4: 26


In [13]:
unique_labels = np.unique(y)
y_encoded = np.unique(y_encoded, axis=0)


In [15]:
index = 26
one_hot_vector = y_encoded[index]

# Find the index of the '1' in the one-hot vector
label_index = np.argmax(one_hot_vector)

# Print the corresponding original label
print("The original label for the one-hot encoded vector at index", index, "is:", unique_labels[label_index])


The original label for the one-hot encoded vector at index 26 is: 9
